In [ ]:
import sqlite3
import pandas as pd
import re

In [ ]:
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return [row[0] for row in conn.execute(query).fetchall()]

def get_columns(conn, table):
    query = f"PRAGMA table_info([{table}])"
    return pd.DataFrame(conn.execute(query).fetchall(), columns=["cid","name","type","notnull","dflt_value","pk"])

def get_fully_null_columns(conn, table, columns):
    """Return columns that have all NULLs."""
    null_cols = []
    cursor = conn.cursor()
    for col in columns:
        query = f"SELECT COUNT(*) FROM [{table}] WHERE [{col}] IS NOT NULL"
        cursor.execute(query)
        count_not_null = cursor.fetchone()[0]
        if count_not_null == 0:
            null_cols.append(col)
    return null_cols

def recreate_table_without_columns(conn, table, cols_to_remove):
    cols_df = get_columns(conn, table)
    cols_to_keep = [col for col in cols_df['name'] if col not in cols_to_remove]

    cols_str = ", ".join([f"[{c}]" for c in cols_to_keep])

    cursor = conn.cursor()
    cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='table' AND name=?;", (table,))
    create_sql = cursor.fetchone()[0]

    pattern = re.compile(r'\((.*)\)', re.DOTALL)
    match = pattern.search(create_sql)
    if not match:
        raise Exception(f"Could not parse CREATE TABLE statement for {table}")
    cols_defs = match.group(1).strip()

    start = 0
    parens = 0
    cols_defs_list = []
    for i, ch in enumerate(cols_defs):
        if ch == '(':
            parens += 1
        elif ch == ')':
            parens -= 1
        elif ch == ',' and parens == 0:
            cols_defs_list.append(cols_defs[start:i].strip())
            start = i+1
    cols_defs_list.append(cols_defs[start:].strip())

    def col_name_from_def(def_str):
        return def_str.split()[0].strip('[]"')

    filtered_cols_defs = [coldef for coldef in cols_defs_list if col_name_from_def(coldef) not in cols_to_remove]

    new_table = table + "_new"
    new_create_sql = f"CREATE TABLE [{new_table}] (\n  " + ",\n  ".join(filtered_cols_defs) + "\n);"

    cursor.execute(new_create_sql)
    cursor.execute(f"INSERT INTO [{new_table}] ({cols_str}) SELECT {cols_str} FROM [{table}];")
    cursor.execute(f"DROP TABLE [{table}];")
    cursor.execute(f"ALTER TABLE [{new_table}] RENAME TO [{table}];")
    conn.commit()


In [ ]:
def clean_null_columns_trim_leading_trailing(db_path):
    conn = sqlite3.connect(db_path)
    tables = get_table_names(conn)

    for table in tables:
        print(f"Processing table: {table}")
        cols_df = get_columns(conn, table)
        columns = cols_df['name'].tolist()

        year_cols = sorted([col for col in columns if col.isdigit()], key=int)

        fully_null_cols = get_fully_null_columns(conn, table, year_cols)

        to_remove = []

        for col in year_cols:
            if col in fully_null_cols:
                to_remove.append(col)
            else:
                break  

        for col in reversed(year_cols):
            if col in fully_null_cols and col not in to_remove:
                to_remove.append(col)
            else:
                break  

        if to_remove:
            print(f" - Removing leading/trailing fully NULL columns: {sorted(to_remove, key=int)}")
            recreate_table_without_columns(conn, table, to_remove)
        else:
            print(" - No leading/trailing fully NULL columns to remove.")

    conn.close()

if __name__ == "__main__":
    db_path = r"C:\IFs\RUNFILES\IFsDataImport.db"
    clean_null_columns_trim_leading_trailing(db_path)
    print("Finished cleaning leading/trailing null-year columns.")
